In [1]:
import torch
import random
import torch.nn.functional as F
import torchvision
import numpy as np
from math import *
import matplotlib.pyplot as plt
from torch.autograd import Variable
from IPython import display
import torch.utils.data as Data
import torch.nn as nn
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator, FormatStrFormatter
%matplotlib notebook
from matplotlib import cm
from scipy.linalg import block_diag
import datetime
from torch.nn.utils import *

In [2]:
def DFT_matrix(N):
    i, j = np.meshgrid(np.arange(N), np.arange(N))
    omega = np.exp( - 2 * pi * 1J / N )
    W = np.power( omega, i * j ) / sqrt(N)
    return np.mat(W)



# 均匀面阵

In [3]:
def freqency_sparse_SV_channel0(Nc,N,sigma_2_alpha,Nt):
    #Nc代表载波数 N代表路径数 sigma_2_alph每条路径能量 sigma_angle角度均匀分布的幅度
    d=0.5
    #print(check)
    Tau = Nc  #最大路径时延
    tau = np.random.rand(1,N)*Tau
    if isinstance(Nt,int):
        n_r = np.mat(range(Nr)).reshape(Nr,1)
        n_t = np.mat(range(Nt)).reshape(Nt,1)
        fai1 = np.random.rand(1,N)*2*pi
        A_t = np.mat(np.zeros([Nt,N],dtype = complex))
        A_r = np.mat(np.ones([1,N],dtype = complex))
        alpha = np.squeeze(np.zeros([1,N],dtype = complex),0)
        H_f = np.array(np.zeros([1,Nc,Nt],dtype = complex))
        for i in range(N):
            A_t[:,i] = np.exp(-2j*pi*0.5*n_t*np.sin(fai1[0,i]))
            aa = (np.random.randn(1,1)+1j*np.random.randn(1,1))*np.sqrt(sigma_2_alpha/2/N)
            alpha[i] = aa[0,0]
    else:
        N_r = Nr
        N_t = Nt[0]*Nt[1]
        fait = np.random.rand(1,N)*2*pi
        fair = np.random.rand(1,N)*2*pi
        theatt = np.random.rand(1,N)*2*pi
        theatr = np.random.rand(1,N)*2*pi
        A_t = np.mat(np.zeros([N_t,N],dtype = complex))
        A_r = np.mat(np.zeros([N_r,N],dtype = complex))
        alpha = np.squeeze(np.zeros([1,N],dtype = complex),0)
        H_f = np.array(np.zeros([N_r,N_t,Nc],dtype = complex))
        n_t1 = np.mat(range(Nt[0])).reshape(Nt[0],1)
        n_t2 = np.mat(range(Nt[1])).reshape(Nt[1],1)
        A_r = np.mat(np.ones([1,N],dtype = complex))
        H_f = np.array(np.zeros([1,Nc,N_t],dtype = complex))
        for i in range(N):
            at1 = np.exp(-2j*pi*0.5*n_t1*np.cos(fait[0,i])*np.sin(theatt[0,i]))
            at2 = np.exp(-2j*pi*0.5*n_t2*np.sin(fait[0,i]))
            A_t[:,i] = np.kron(at1,at2)

            aa = (np.random.randn(1,1)+1j*np.random.randn(1,1))*np.sqrt(sigma_2_alpha/2/N)  
            alpha[i] = aa[0,0]
                
                
    for k in range(Nc):
        P = np.squeeze(alpha*np.exp(-1j*2*pi*tau*k/Nc),0);
        P = np.diag(P)
        H_f[:,k,:] = np.dot(np.dot(A_r,P),A_t.H)
        
    return H_f,A_t,alpha

In [4]:
Nc = 32
sigma_2_alpha = 1
Nt = [8,8]
N_t = Nt[0]*Nt[1]
Nr = 1

B = 30
D = 1000; #角度采样点数
L = 8

SNR_dB = 10
K = 4
snr =  10**(SNR_dB/10)/K

N_BATCH_train = 400
N_BATCH_test  = 80
BATCH_SIZE = 256
N_H_train = N_BATCH_train*BATCH_SIZE
N_H_test = N_BATCH_test*BATCH_SIZE

In [5]:
for N in range(1,9):
    H_torch = torch.zeros([BATCH_SIZE*N_BATCH_train,K,Nc,N_t*2])
    for i in range(N_BATCH_train):
        H = np.zeros([BATCH_SIZE,K,Nc,N_t],dtype=complex) #第0个维度是样本 第1个维度是用户，第2个维度是子载波，第3个维度是天线
        for j in range(BATCH_SIZE):
            for k in range(K):
                H_f,A_t,alpha = freqency_sparse_SV_channel0(Nc,N,sigma_2_alpha,Nt)
                H[j,k,:,:] = H_f
    
        H_torch[(i*BATCH_SIZE):((i+1)*BATCH_SIZE),:,:,0:N_t] = torch.from_numpy(np.real(H))
        H_torch[(i*BATCH_SIZE):((i+1)*BATCH_SIZE),:,:,N_t:2*N_t] = torch.from_numpy(np.imag(H))
        print(i)
    torch.save(H_torch,'data/H_train_UPA'+str(N)+'Lp_1.pt')
    
    H_torch = torch.zeros([BATCH_SIZE*N_BATCH_train,K,Nc,N_t*2])
    for i in range(N_BATCH_train):
        H = np.zeros([BATCH_SIZE,K,Nc,N_t],dtype=complex) #第0个维度是样本 第1个维度是用户，第2个维度是子载波，第3个维度是天线
        for j in range(BATCH_SIZE):
            for k in range(K):
                H_f,A_t,alpha = freqency_sparse_SV_channel0(Nc,N,sigma_2_alpha,Nt)
                H[j,k,:,:] = H_f
    
        H_torch[(i*BATCH_SIZE):((i+1)*BATCH_SIZE),:,:,0:N_t] = torch.from_numpy(np.real(H))
        H_torch[(i*BATCH_SIZE):((i+1)*BATCH_SIZE),:,:,N_t:2*N_t] = torch.from_numpy(np.imag(H))
        print(i)
    torch.save(H_torch,'data/H_train_UPA'+str(N)+'Lp_2.pt')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

398
399
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
27

In [6]:
for N in range(1,9):
    H_torch = torch.zeros([BATCH_SIZE*N_BATCH_test,K,Nc,N_t*2])
    for i in range(N_BATCH_test):
        H = np.zeros([BATCH_SIZE,K,Nc,N_t],dtype=complex) #第0个维度是样本 第1个维度是用户，第2个维度是子载波，第3个维度是天线
        for j in range(BATCH_SIZE):
            for k in range(K):
                H_f,A_t,alpha = freqency_sparse_SV_channel0(Nc,N,sigma_2_alpha,Nt)
                H[j,k,:,:] = H_f
    
        H_torch[(i*BATCH_SIZE):((i+1)*BATCH_SIZE),:,:,0:N_t] = torch.from_numpy(np.real(H))
        H_torch[(i*BATCH_SIZE):((i+1)*BATCH_SIZE),:,:,N_t:2*N_t] = torch.from_numpy(np.imag(H))
        print(i)
    torch.save(H_torch,'data/H_test_UPA'+str(N)+'Lp.pt')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [7]:

H_torch = torch.zeros([BATCH_SIZE*N_BATCH_train,K,Nc,N_t*2])
for i in range(N_BATCH_train):
    H = np.zeros([BATCH_SIZE,K,Nc,N_t],dtype=complex) #第0个维度是样本 第1个维度是用户，第2个维度是子载波，第3个维度是天线
    for j in range(BATCH_SIZE):
        N = np.int(np.random.rand(1)*8)+1
        for k in range(K):
            H_f,A_t,alpha = freqency_sparse_SV_channel0(Nc,N,sigma_2_alpha,Nt)
            H[j,k,:,:] = H_f
    
    H_torch[(i*BATCH_SIZE):((i+1)*BATCH_SIZE),:,:,0:N_t] = torch.from_numpy(np.real(H))
    H_torch[(i*BATCH_SIZE):((i+1)*BATCH_SIZE),:,:,N_t:2*N_t] = torch.from_numpy(np.imag(H))
    print(i)
torch.save(H_torch,'data/H_train_UPA'+str(-1)+'Lp_1.pt')

H_torch = torch.zeros([BATCH_SIZE*N_BATCH_train,K,Nc,N_t*2])
for i in range(N_BATCH_train):
    H = np.zeros([BATCH_SIZE,K,Nc,N_t],dtype=complex) #第0个维度是样本 第1个维度是用户，第2个维度是子载波，第3个维度是天线
    for j in range(BATCH_SIZE):
        N = np.int(np.random.rand(1)*8)+1
        for k in range(K):
            H_f,A_t,alpha = freqency_sparse_SV_channel0(Nc,N,sigma_2_alpha,Nt)
            H[j,k,:,:] = H_f
    
    H_torch[(i*BATCH_SIZE):((i+1)*BATCH_SIZE),:,:,0:N_t] = torch.from_numpy(np.real(H))
    H_torch[(i*BATCH_SIZE):((i+1)*BATCH_SIZE),:,:,N_t:2*N_t] = torch.from_numpy(np.imag(H))
    print(i)
torch.save(H_torch,'data/H_train_UPA'+str(-1)+'Lp_2.pt')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
H_torch = torch.zeros([BATCH_SIZE*N_BATCH_test,K,Nc,N_t*2])
for i in range(N_BATCH_test):
    H = np.zeros([BATCH_SIZE,K,Nc,N_t],dtype=complex) #第0个维度是样本 第1个维度是用户，第2个维度是子载波，第3个维度是天线
    for j in range(BATCH_SIZE):
        N = np.int(np.random.rand(1)*8)+1
        for k in range(K):
            H_f,A_t,alpha = freqency_sparse_SV_channel0(Nc,N,sigma_2_alpha,Nt)
            H[j,k,:,:] = H_f
    
    H_torch[(i*BATCH_SIZE):((i+1)*BATCH_SIZE),:,:,0:N_t] = torch.from_numpy(np.real(H))
    H_torch[(i*BATCH_SIZE):((i+1)*BATCH_SIZE),:,:,N_t:2*N_t] = torch.from_numpy(np.imag(H))
    print(i)
torch.save(H_torch,'data/H_test_UPA'+str(-1)+'Lp.pt')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79


In [12]:
import scipy.io as io

H = np.zeros([BATCH_SIZE,K,Nc,N_t],dtype=complex) #第0个维度是样本 第1个维度是用户，第2个维度是子载波，第3个维度是天线
for j in range(BATCH_SIZE):
    for k in range(K):
        H_f,A_t,alpha = freqency_sparse_SV_channel0(Nc,N,sigma_2_alpha,Nt)
        H[j,k,:,:] = H_f
H_torch = torch.zeros([BATCH_SIZE,K,Nc,N_t*2])
H_torch[:,:,:,0:N_t] = torch.from_numpy(np.real(H))
H_torch[:,:,:,N_t:2*N_t] = torch.from_numpy(np.imag(H))
#torch.save(H_torch,'data/H_test_UPA.pt')
print(Nt)


dataNew = 'D:\python_project\model_quan\data\H_UPA.mat'
io.savemat(dataNew, {'H_UPA':H})

[8, 8]
